## Which annotation files to use for FC_directed?

In [3]:
%cd ~/projects/funcoup_directed/

/Users/davidebuzzao/Projects/funcoup_directed


In [4]:
import pandas as pd, pickle
genome = 'GRCh38'
organism = 'human'

### ENCODE

The human genome is estimated to encode approximately 2000 TFs, which operate programs that change cellular states by binding to proxy or distal cis-regulatory elements (CREs) for a set of target genes. Interactions between TFs and CREs of target genes are generally detected by DNA-binding experiments such as chromatin immunoprecipitation (ChIP), which is often followed by microarray analysis (ChIP-chip) or deep-sequencing analysis (ChIP-seq). 
ChIP-seq is therefore one of the methods used to analyze protein interactions with DNA. It combines chromatin immunoprecipitation with DNA sequencing to infer the possible binding sites of DNA-associated proteins. ChIP-seq data are available on the Encyclopedia of DNA Elements (ENCODE) portal (Davis et al., 2018) and can be used to link transcription factors with their targets. The ENCODE project, in fact, has been formed by the NHGRI in 2003 with the aim of bringing together a variety of experimental and computational labs to identify biochemically active regions in the human and mouse genomes utilizing a variety of HT approaches. Multiple pipelines has been developed by the ENCODE consortium, a couple of which (in particular) to study two different classes of protein-chromatin interactions: 
1. the pipeline for transcription factor ChIP-seq (TF ChIP-seq) 
2. the pipeline for histone ChIP-seq. 

Both pipelines are maintained in the ENCODE Data Coordinating Center (DCC) GitHub repository (https://github.com/ENCODE-DCC/chip-seq-pipeline2).  

For what concerns the transcription factor ChIP-seq (TF ChIP-seq) pipeline, it is suitable for proteins that are expected to bind in a punctate manner, such as to specific DNA sequences or specific chromatin configurations. In these assays, the IP target is typically a known or putative transcription factor or chromatin remodeller but can also be an RNA-binding protein or other DNA- or chromatin-specific factor. For the FunCoup purpose, everything but transcription factors’ assays is filtered out, remaining with 2715 TF-ChIP-seq experiments on Homo sapiens, Drosophila melanogaster, Caenorhabditis elegans and Mus musculus

In [8]:
%wget -O data/evidences/encode/human/metadata.tsv $(head -n +1 data/evidences/encode/human/files.txt)

UsageError: Line magic function `%wget` not found.


In [5]:
metadata = pd.read_csv("data/evidences/encode/human/metadata.tsv", sep="\t")
metadata

,File accession,File format,File type,File format type,Output type,Experiment accession,Assay,Biosample term id,Biosample term name,Biosample type,...,Assembly,Genome annotation,Platform,Controlled by,File Status,s3_uri,Audit WARNING,Audit INTERNAL_ACTION,Audit NOT_COMPLIANT,Audit ERROR
0,ENCFF827IYA,bam,bam,NaN,unfiltered alignments,ENCSR445ACU,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2018/02/21/e515ec26-4792-40...,NaN,NaN,NaN,NaN
1,ENCFF393IBD,bam,bam,NaN,unfiltered alignments,ENCSR445ACU,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2018/02/21/924de693-bf20-4d...,NaN,NaN,NaN,NaN
2,ENCFF123GSL,bigWig,bigWig,NaN,signal p-value,ENCSR445ACU,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2018/02/26/48e60a16-f62a-4f...,NaN,NaN,NaN,NaN
3,ENCFF174SPG,bed narrowPeak,bed,narrowPeak,peaks and background as input for IDR,ENCSR445ACU,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2018/02/26/4445d47e-1624-4b...,NaN,NaN,NaN,NaN
4,ENCFF198ATT,bigBed narrowPeak,bigBed,narrowPeak,peaks and background as input for IDR,ENCSR445ACU,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2018/02/26/5a89254e-fec5-4e...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60602,ENCFF724MHH,bigBed narrowPeak,bigBed,narrowPeak,peaks and background as input for IDR,ENCSR000BLE,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2017/07/07/422dcd55-6fff-43...,NaN,NaN,NaN,NaN
60603,ENCFF722GFS,bed narrowPeak,bed,narrowPeak,conservative IDR thresholded peaks,ENCSR000BLE,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2017/07/07/eb2b9798-4787-4d...,NaN,NaN,NaN,NaN
60604,ENCFF894NYR,bigBed narrowPeak,bigBed,narrowPeak,optimal IDR thresholded peaks,ENCSR000BLE,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2017/07/07/a7bb1f7a-4bd3-41...,NaN,NaN,NaN,NaN
60605,ENCFF506PCG,bigBed narrowPeak,bigBed,narrowPeak,conservative IDR thresholded peaks,ENCSR000BLE,ChIP-seq,EFO:0001187,HepG2,cell line,...,hg19,NaN,NaN,NaN,released,s3://encode-public/2017/07/07/51d513dd-7bf6-4b...,NaN,NaN,NaN,NaN


The pipeline takes both ChIP-seq reads (from paired-end, stranded or single-end, unstranded libraries) and a set of reference fastas as inputs, and outputs unfiltered and filtered alignment files in bam format (from sam: the Sequence Alignment/Mapping format is a text-based format for storing read alignments against reference sequences and it is interconvertible with the binary bam format) from the appropriate genome assembly. When multiple fastqs are generated from a single biological replicate (in multiple sequencing runs, for example), they are concatenated before mapping. With no exception: the read length should be a minimum of 50 base pairs; the sequencing platform used should be indicated and replicates should match in terms of read length and run type. Pipeline files are mapped to either the GRCh38 or mm10 sequences. Also, data that were originally collected and mapped in a non-standard fashion to earlier assemblies in prior project phases, has now been reprocessed using the ENCODE unified processing pipelines both onto a consistent hg19 (GRCh37) reference genome assembly and the updated GRCh38 reference. Similarly, mouse data previously mapped to mm9 have also been reprocessed onto the latest assembly, mm10. Reference files used in the ENCODE processing pipeline are retrievable at https://www.encodeproject.org/references/ENCSR425FOI/.

Metadata in ENCODE is captured in JavaScript Object Notation (JSON). Metadata tables for datasets can be either downloaded in (.tsv) format via the web interface or obtained programmatically through the REST API (https://www.encodeproject.org/help/rest-api/). Here the subset of the information I consider necessary to be extracted from metadata and the way to retrieve them.


In [ ]:
head -n +1 data/evidences/encode/human/metadata.tsv | cut -f 1,2,5,6,7,11,21,41,46 > data/evidences/encode/human/metadata_bam.tsv

In [ ]:
grep bam data/evidences/encode/human/metadata.tsv | grep -v unfiltered | grep '\bGRCh38\b' | cut -f 1,2,5,6,7,11,21,41,46 >> data/evidences/encode/human/metadata_bam.tsv

In [12]:
met_bam = pd.read_csv("data/evidences/encode/human/metadata_bam.tsv", sep="\t")
met_bam

,File accession,File format,Output type,Experiment accession,Assay,Biosample organism,Experiment target,Size,Assembly
0,ENCFF908SPX,bam,alignments,ENCSR445ACU,ChIP-seq,Homo sapiens,SOX13-human,1106836211,GRCh38
1,ENCFF010EFV,bam,alignments,ENCSR445ACU,ChIP-seq,Homo sapiens,SOX13-human,1021640457,GRCh38
2,ENCFF582CJF,bam,alignments,ENCSR000BGM,ChIP-seq,Homo sapiens,USF1-human,637033775,GRCh38
3,ENCFF270TGL,bam,alignments,ENCSR000BGM,ChIP-seq,Homo sapiens,USF1-human,999909157,GRCh38
4,ENCFF803MUP,bam,alignments,ENCSR634OAQ,ChIP-seq,Homo sapiens,CTCF-human,4302449535,GRCh38
...,...,...,...,...,...,...,...,...,...
2976,ENCFF686DDV,bam,alignments,ENCSR000DKR,ChIP-seq,Homo sapiens,CTCF-human,582699200,GRCh38
2977,ENCFF779BHG,bam,alignments,ENCSR000DKR,ChIP-seq,Homo sapiens,CTCF-human,1019590180,GRCh38
2978,ENCFF205XEH,bam,alignments,ENCSR000AUF,ChIP-seq,Homo sapiens,CTCF-human,1292831532,GRCh38
2979,ENCFF172IMH,bam,alignments,ENCSR000BLE,ChIP-seq,Homo sapiens,FOXA1-human,497461256,GRCh38


For the FunCoup purpose, I checked the frequency of TFs in H. sapiens in the retrievable ChIP-seq experiments. No TF is studied as much as CTCF. This is probably due to the fact the transcriptional repressor is involved in many cellular processes (including transcriptional regulation, insulator activity, V(D)J recombination and regulation of chromatin architecture) and therefore the protein could be subjected to various studies by laboratories working in different fields. The TFs that have been under investigation only once are 275/489 in human and 10/47 in mouse. 

Here I extract unique `Experiment accession` and exclude replication of same ChIP-Seq experiment.

In [ ]:
tail -n +2  data/evidences/encode/human/metadata_bam.tsv | cut -f 4 | uniq > data/evidences/encode/human/exp_accession.id

In [ ]:
head -n +1 data/evidences/encode/human/metadata_bam.tsv > data/evidences/encode/human/metadata_bam.no_rep.tsv

In [ ]:
for i in `cat data/evidences/encode/human/exp_accession.id`; do grep -m 1 $i data/evidences/encode/human/metadata_bam.tsv; done >> data/evidences/encode/human/metadata_bam.no_rep.tsv

In [7]:
metadata_bam_norep = pd.read_csv("data/evidences/encode/human/metadata_bam.no_rep.tsv", sep="\t")
metadata_bam_norep

,ENCFF908SPX,bam,alignments,ENCSR445ACU,ChIP-seq,Homo sapiens,SOX13-human,1106836211,GRCh38
0,ENCFF582CJF,bam,alignments,ENCSR000BGM,ChIP-seq,Homo sapiens,USF1-human,637033775,GRCh38
1,ENCFF803MUP,bam,alignments,ENCSR634OAQ,ChIP-seq,Homo sapiens,CTCF-human,4302449535,GRCh38
2,ENCFF654KYF,bam,alignments,ENCSR000DQD,ChIP-seq,Homo sapiens,CTCF-human,819184321,GRCh38
3,ENCFF472PUP,bam,alignments,ENCSR000DKN,ChIP-seq,Homo sapiens,CTCF-human,879838417,GRCh38
4,ENCFF449ZQR,bam,alignments,ENCSR094ZCF,ChIP-seq,Homo sapiens,CEBPG-human,1268211309,GRCh38
...,...,...,...,...,...,...,...,...,...
1513,ENCFF949KAW,bam,redacted alignments,ENCSR336PTS,ChIP-seq,Homo sapiens,CTCF-human,650522945,GRCh38
1514,ENCFF893HMO,bam,alignments,ENCSR000DXQ,ChIP-seq,Homo sapiens,CTCF-human,365418600,GRCh38
1515,ENCFF686DDV,bam,alignments,ENCSR000DKR,ChIP-seq,Homo sapiens,CTCF-human,582699200,GRCh38
1516,ENCFF205XEH,bam,alignments,ENCSR000AUF,ChIP-seq,Homo sapiens,CTCF-human,1292831532,GRCh38


Here I extract uniq `Experiment target` and count with which frequency the TF is under study.

In [ ]:
tail -n +2 data/evidences/encode/human/metadata_bam.tsv | cut -f 7 | cut -d '-' -f 1 | sort -u > data/evidences/encode/human/tf.id

In [ ]:
for i in `cat data/evidences/encode/human/tf.id`; do paste <(echo $i) <(grep -c $i data/evidences/encode/human/metadata_bam.no_rep.tsv); done | sort -grk 2 > data/evidences/encode/human/tf.freq

In [23]:
tf_freq = pd.read_csv("data/evidences/encode/human/tf.freq", sep="\t", names=['exp target', 'freq'], header=None)
print('Number of TFs:\t%i' %len(tf_freq))
tf_freq.loc[0:10,:]

Number of TFs:	489


,exp target,freq
0,CTCF,308
1,EP300,50
2,JUN,42
3,ZNF2,41
4,NR3C1,28
5,ZNF3,26
6,FOS,26
7,REST,22
8,CEBPB,22
9,MYC,18


I download the gene annotation file from **www.encodeproject.org/files**. Extract only protein coding genes and get name of the gene and gene's coordinates. 

In [ ]:
wget -O data/evidences/encode/human/GRCh38_GENCODE_V29.gtf.gz https://www.encodeproject.org/files/ENCFF159KBI/@@download/ENCFF159KBI.gtf.gz

In [ ]:
zcat < data/evidences/encode/human/GRCh38_GENCODE_V29.gtf.gz \
    | awk 'BEGIN{FS="\t"}{split($9,a,";"); if(($3~"gene") && (a[2]~"protein_coding")) print a[3]"\t"$1"\t"$7"\t"$4"\t"$5}' \
    | sed 's/gene_name "//; s/"//g' | grep -v random | grep -v chrUn > data/evidences/encode/human/GRCh38_GENCODE_V29.table.gtf

In [24]:
grch38_geneanno = pd.read_csv("data/evidences/encode/human/GRCh38_GENCODE_V29.table.gtf", sep="\t", header=None, names=['gene', 'chr', 'pol', 'sta', 'end'])
grch38_geneanno

,gene,chr,pol,sta,end
0,OR4F5,chr1,+,65419,71585
1,OR4F29,chr1,-,450703,451697
2,OR4F16,chr1,-,685679,686673
3,SAMD11,chr1,+,923928,944581
4,NOC2L,chr1,-,944204,959309
...,...,...,...,...,...
19935,MT-ND4L,chrM,+,10470,10766
19936,MT-ND4,chrM,+,10760,12137
19937,MT-ND5,chrM,+,12337,14148
19938,MT-ND6,chrM,-,14149,14673


I modify the `GRCh38_GENCODE_V29.table.gtf` chromosomes' starts and ends to finally work with TIP method.

In [ ]:
awk 'BEGIN{FS="\t"} \
    {if($3=="+") {start=($4-10000); end=($4+10000); print $1"\t"$2"\t"$3"\t"start"\t"end;} \
    else if($3=="-") {start=($5+10000); end=($5-10000); print $1"\t"$2"\t"$3"\t"end"\t"start;}}'\
    data/evidences/encode/human/GRCh38_GENCODE_V29.table.gtf > data/evidences/encode/human/GRCh38_GENCODE_V29.TIP20000.gtf

In [25]:
grch38_geneanno = pd.read_csv("data/evidences/encode/human/GRCh38_GENCODE_V29.TIP20000.gtf", sep="\t", header=None, names=['gene', 'chr', 'pol', 'sta', 'end'])
grch38_geneanno

,gene,chr,pol,sta,end
0,OR4F5,chr1,+,55419,75419
1,OR4F29,chr1,-,441697,461697
2,OR4F16,chr1,-,676673,696673
3,SAMD11,chr1,+,913928,933928
4,NOC2L,chr1,-,949309,969309
...,...,...,...,...,...
19935,MT-ND4L,chrM,+,470,20470
19936,MT-ND4,chrM,+,760,20760
19937,MT-ND5,chrM,+,2337,22337
19938,MT-ND6,chrM,-,4673,24673


## RegNetwork

**RegNetwork** is a database that stores knowledge-based genome-wide regulatory networks (Liu et al., 2015). So far it has been built for organisms as human and mouse by integrating 25 data sources. A comprehensive set of interplays among TFs, miRNAs and target genes are collected and reorganized for public access. Specifically, the mouse regulatory network contains 20738 nodes and 323636 edges, consisting of 1328 TFs, 1290 miRNAs and 18120 target genes. 
It should be noted that, besides the experimentally observed or discovered regulatory relationships that have been curated in public databases such as TRED and KEGG, RegNetwork integrates predictions based on TF binding site (TFBS) information for TF-gene and considers protein-protein interactions (PPIs) from HPRD, BioGrid, IntAct, KEGG and STRING as putative gene regulations when PPI pairs contain at least one TF (self-regulations are also identified in this case when TF-TF interactions is observed). It is a fortune that a degree of confidence for each of the regulatory interactions, by using a three-level labelling approach (i.e. a high, medium or low confidence), has been added. More specifically, the experiment-validated regulations are tagged with the label high confidence; the predictions made by only one algorithm/method are tagged with low confidence; and the rest are tagged with medium confidence. Although these confidence levels are not reported in the downloadable files from the download section of the website, one can use their search engine to filter out predictions and what has low or medium confidence. When one retrieves only the experiment-validated regulations with label high confidence, the coverage of the database decreases a lot.

In [26]:
report = pd.read_csv("data/db/regnet/human/export_Tue_Apr_07_20_14_32_UTC_2020.csv", sep=",")
report

,regulator_symbol,regulator_id,target_symbol,target_id,database,evidence,confidence
0,ABL1,25,SHC3,53358,kegg,Experimental,High
1,ABL1,25,STAT5B,6777,kegg,Experimental,High
2,ABL1,25,CBLB,868,kegg,Experimental,High
3,ABL1,25,CBLC,23624,kegg,Experimental,High
4,ABL1,25,CD55,1604,kegg,Experimental,High
...,...,...,...,...,...,...,...
9721,ZIC2,7546,GLI2,2736,"hprd,kegg",Experimental,High
9722,ZIC2,7546,GLI1,2735,"hprd,kegg",Experimental,High
9723,ZIC2,7546,GLI3,2737,"hprd,kegg",Experimental,High
9724,ZNF274,10782,NGFR,4804,"hprd,kegg",Experimental,High


In [37]:
with open("data/db/regnet/human/regnet_db.pkl", 'rb') as f1:
    regnet_db = pickle.load(f1)
regnet_db['ZNF274']

{'Activation': [],
 'Repression': [],
 'Unknown': ['NGFR', 'TRAF6', 'NGFR', 'TRAF6', 'NGFR', 'TRAF6']}

## TRRUSTv2
**TRRUST** (Transcriptional Regulatory Relationships Unraveled by Sentence-based Text-mining) is a database of reference TF–target regulatory interactions identified via the manual curation of Medline abstracts (Han et al., 2015). Sentence-based text mining and prioritization of pre-candidate sentences is conducted to reduce the cost-effective literature curation by differentiating sentences for their likelihood of containing information regarding TF–gene interactions.

The current version of TRRUSTv2 contains 6490 TF–target interactions for 827 mouse's TFs.  Most importantly, a majority of the interactions have annotations for mode-of-regulation (i.e., activation or repression).  If a sentence suggested a mode-of-regulation, i.e., either activation or repression, for the given TF-gene regulatory interaction, the authors collected and deposited the information along with the given TF-gene interaction in the database. If there was no indication in either the given sentence or its associated abstract, then the term unknown for the mode-of-regulation of the TF-gene regulatory interaction was assigned. Currently 8972 (59.8%) TF-gene interactions, over both human and mouse 14996 regulatory relationships, are known for mode of regulation.

This exclusive to TRRUSTv2 could be useful if one would like to add annotation to new directed FunCoup links being mode-of-regulation important in interpreting the phenotype effects of TF dysregulation. 

In [30]:
report = pd.read_csv("data/db/trrust/human/trrust_rawdata.human.tsv", sep="\t", header=None, names=['regulator_symbol', 'target_symbol', 'mode_of_regulation', 'PMID'])
report

,regulator_symbol,target_symbol,mode_of_regulation,PMID
0,AATF,BAX,Repression,22909821
1,AATF,CDKN1A,Unknown,17157788
2,AATF,KLK3,Unknown,23146908
3,AATF,MYC,Activation,20549547
4,AATF,TP53,Unknown,17157788
...,...,...,...,...
9391,ZNF76,CDKN1A,Repression,15280358
9392,ZNF76,PCYT1A,Activation,14702349
9393,ZNF76,TALDO1,Unknown,14702349
9394,ZNRD1,ABCB1,Activation,16373708


In [33]:
with open("data/db/trrust/human/trrust_db.pkl", 'rb') as f1:
    trrust_db = pickle.load(f1)
trrust_db['AATF']

{'Activation': [],
 'Repression': [],
 'Unknown': ['BAX', 'CDKN1A', 'KLK3', 'MYC', 'TP53']}

I get from `https://ftp.ncbi.nlm.nih.gov/genomes/` the information about name and lenghts of chromosomes. 

In [50]:
if organism == 'human':
    chr_nam = ['chr' + str(i) for i in range(1,23)] + ['chrX', 'chrY', 'chrM']
    if genome == 'GRCh38': 
        chr_len = [ 248956422, 242193529, 198295559, 190214555, 181538259, 170805979, 159345973, 
                    145138636, 138394717, 133797422, 135086622, 133275309, 114364328, 107043718, 
                    101991189, 90338345, 83257441, 80373285, 58617616, 64444167, 46709983, 50818468, 
                    156040895, 57227415, 16569 ]

dictio = {}
for i in range(len(chr_nam)):
    dictio[chr_nam[i]] = dictio.get(chr_nam[i], chr_len[i])
pd.DataFrame(dictio.items(), columns=['chr','length']).T

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
chr,chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,...,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX,chrY,chrM
length,248956422,242193529,198295559,190214555,181538259,170805979,159345973,145138636,138394717,133797422,...,90338345,83257441,80373285,58617616,64444167,46709983,50818468,156040895,57227415,16569
